In [51]:
import pandas as pd
import os
import numpy as np

In [99]:
df = pd.read_csv(os.path.join("data", "HDFCBANK_SMC_merged.csv"))

In [100]:
df.columns

Index(['index', 'symbol', 'date', 'open', 'high', 'low', 'close',
       'adjusted_close', 'volume', 'shl_HighLow', 'shl_Level', 'bos_BOS',
       'bos_CHOCH', 'bos_Level', 'bos_BrokenIndex', 'ob_OB', 'ob_Top',
       'ob_Bottom', 'ob_OBVolume', 'ob_MitigatedIndex', 'ob_Percentage',
       'fvg_FVG', 'fvg_Top', 'fvg_Bottom', 'fvg_MitigatedIndex',
       'liq_Liquidity', 'liq_Level', 'liq_End', 'liq_Swept'],
      dtype='object')

In [6]:
df = df.drop(['index', 'symbol', 'sma', 'diff', 'diff/avg_diff', 'momentum',
       'delta_m', 'CF', 'DDF', 'TSR', 'Class',  'ob_OB',
       'ob_Top', 'ob_Bottom', 'ob_OBVolume', 'ob_MitigatedIndex',
       'ob_Percentage', 'fvg_FVG', 'fvg_Top', 'fvg_Bottom',
       'fvg_MitigatedIndex', 'liq_Liquidity', 'liq_Level', 'liq_End',
       'liq_Swept'], axis=1)

In [101]:
df

,index,symbol,date,open,high,low,close,adjusted_close,volume,shl_HighLow,...,ob_MitigatedIndex,ob_Percentage,fvg_FVG,fvg_Top,fvg_Bottom,fvg_MitigatedIndex,liq_Liquidity,liq_Level,liq_End,liq_Swept
0,0,HDFCBANK.NSE,2020-01-01,1276.10,1288.00,1263.60,1268.5,595.3604,20666414,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,HDFCBANK.NSE,2020-01-06,1260.00,1286.90,1236.00,1281.0,601.2271,55707686,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,HDFCBANK.NSE,2020-01-13,1282.70,1296.50,1271.90,1277.4,599.5375,51194520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,HDFCBANK.NSE,2020-01-20,1304.86,1304.86,1231.00,1245.8,584.7063,78872240,NaN,...,NaN,NaN,-1.0,1271.9,1242.0,5.0,NaN,NaN,NaN,NaN
4,4,HDFCBANK.NSE,2020-01-27,1235.00,1242.00,1191.36,1192.5,559.6904,79206598,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,293,HDFCBANK.NSE,2025-08-11,1975.90,1997.60,1967.50,1989.5,994.7500,53132768,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,294,HDFCBANK.NSE,2025-08-18,2010.00,2026.60,1957.10,1965.5,982.7500,73860814,NaN,...,NaN,NaN,-1.0,1967.5,988.7,0.0,NaN,NaN,NaN,NaN
295,295,HDFCBANK.NSE,2025-08-25,980.10,988.70,950.10,953.4,953.4000,74697843,NaN,...,NaN,NaN,-1.0,1957.1,967.5,297.0,NaN,NaN,NaN,NaN
296,296,HDFCBANK.NSE,2025-09-01,949.60,967.50,940.00,962.6,962.6000,79657913,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
bullish_pattern = [-1, 1, -1, 1] # Bullish CoC+
bearish_pattern = [1, -1, 1, -1] # Bearish CoC+

### Bullish CoC+

*swing_low* -> *lower swing_high* ($bearish$) -> *higher swing_low* ($CoC+$) -> *higher swing_high* ($bullish$)

### Bearish CoC+

*swing_high* -> *higher swing_low* ($bullish$) -> *lower swing_high* ($CoC+$) -> *lower swing_low* ($bearish$)

In [111]:
l = 20 # Lookback
df['CoC+'] = np.zeros(len(df))
df['level'] = np.zeros(len(df))
for i in range(l, len(df)):
    # Use series to retain original indices for swings
    shl_series = df['shl_HighLow'].iloc[i-l: i].dropna()
    level_series = df['shl_Level'].iloc[i-l: i].dropna()
    # Use closes at the swing indices for CoC comparisons
    close_series = df.loc[shl_series.index, 'close']
    hist = shl_series.to_list()
    levels = level_series.to_list()
    closes = close_series.to_list()
    # Need at least 5 swings to evaluate the pattern logic safely
    if len(hist) < 5:
        continue
    if hist[-4:] == bullish_pattern:
        # Check Break of Structure with bearish expectancy and bullish CoC
        if (levels[-5] > levels[-3]) and (levels[-4] < levels[-2]) and (levels[-3] < levels[-1]):
            # Place marker at the second most recent swing's actual row index
            print(levels)
            print(hist)
            swing_index = shl_series.index[-2]
            df.loc[swing_index, 'CoC+'] = 1
            df.loc[swing_index, 'level'] = close_series.iloc[-2]
    if hist[-4:] == bearish_pattern:
        # Check Break of Structure with bullish expectancy and bearish CoC
        if (levels[-5] < levels[-3]) and (levels[-4] > levels[-2]) and (levels[-3] > levels[-1]):
            # Place marker at the second most recent swing's actual row index
            swing_index = shl_series.index[-2]
            df.loc[swing_index, 'CoC+'] = -1
            df.loc[swing_index, 'level'] = close_series.iloc[-2]

[1157.96, 993.0, 1148.8, 1025.0, 1464.4]
[1.0, -1.0, 1.0, -1.0, 1.0]
[1157.96, 993.0, 1148.8, 1025.0, 1464.4]
[1.0, -1.0, 1.0, -1.0, 1.0]
[1880.0, 1624.3, 1767.0, 1661.1, 1978.9]
[1.0, -1.0, 1.0, -1.0, 1.0]


In [ ]:
l = 20 # Lookback
df['CoC+'] = np.zeros(len(df))
for i in range(l, len(df)):
    index = df.index[i]  # Get the actual index
    hist = df['shl_HighLow'].iloc[i-l: i].dropna().to_list()
    levels = df['shl_Level'].iloc[i-l: i].dropna().to_list()
    if len(hist)<5:
        continue
    if hist[-4:] == bullish_pattern:
        # Check Break of Structure with bearish expectancy and bullish CoC
        if (levels[-5] > levels[-3]) and (levels[-4] < levels[-2]) and (levels[-3] < levels[-1]):
            df.loc[i, 'CoC+'] = 1
            df.loc[i, 'level'] = levels[-2]
    if hist[-4:] == bearish_pattern:
        # Check Break of Structure with bullish expectancy and bearish CoC
        if (levels[-5] < levels[-3]) and (levels[-4] > levels[-2]) and (levels[-3] > levels[-1]):
            df.loc[i, 'CoC+'] = -1
            df.loc[i, 'level'] = levels[-2]


In [73]:
df['CoC+'].value_counts()

CoC+
 0.0    282
-1.0     10
 1.0      5
Name: count, dtype: int64

In [94]:
df.to_csv("example.csv", index=False)

In [76]:
import plotly.graph_objects as go

In [98]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Assuming you already have your dataframe 'df' with the CoC+ signals
# Let's create a visualization

# Create subplots with 2 rows: one for price and CoC+ signals, one for volume (optional)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                   vertical_spacing=0.03, 
                   subplot_titles=('Price with CHoCH+ Signals', 'Volume'),
                   row_width=[0.2, 0.7])

# Add candlestick chart
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='Price'),
              row=1, col=1)

# Add bullish CHoCH+ signals (value = 1)
bullish_signals = df[df['CoC+'] == 1]
fig.add_trace(go.Scatter(x=bullish_signals.index,
                         y=bullish_signals['level'],
                         mode='markers',
                         marker=dict(color='green', size=10, symbol='triangle-up'),
                         name='Bullish CHoCH+',
                         hovertemplate='Bullish CHoCH+<br>Time: %{x}<br>Level: %{y}<extra></extra>'),
              row=1, col=1)

# Add bearish CHoCH+ signals (value = -1)
bearish_signals = df[df['CoC+'] == -1]
fig.add_trace(go.Scatter(x=bearish_signals.index,
                         y=bearish_signals['level'],
                         mode='markers',
                         marker=dict(color='red', size=10, symbol='triangle-down'),
                         name='Bearish CHoCH+',
                         hovertemplate='Bearish CHoCH+<br>Time: %{x}<br>Level: %{y}<extra></extra>'),
              row=1, col=1)

# Add volume bar chart
fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume'), row=2, col=1)

# Update layout
fig.update_layout(
    title='Price Chart with CHoCH+ Signals',
    yaxis_title='Price',
    xaxis_title='Date',
    template='plotly_white',
    height=800,
    showlegend=True
)

# Update x-axis rangeslider
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_xaxes(rangeslider_visible=True, row=2, col=1)

fig.write_html("example.html")